In [1]:
using LinearAlgebra
using Pipe
using QuantEcon
using Roots
using Interpolations

In [46]:
nt = 3000;
nt1 = 500; # Number of Burn in 
nh = 2000; # Number of agents for Simulation

In [44]:
NK = 16
N_Age = 81
NX = 51
NA = 51
NPB = 25
NP = 2
NU = 2
NE = 2
const beta = 0.99
const F_cost = 0.06
PB_min = 0.9523809523809523
PB_max = 1.0309278350515465
PB = range(PB_min,PB_max, length = NPB) |> collect ;
Kgrid = range(4.5, 21 , length = NK) |> collect ;
labmda = 0.4 
const rho_1 = 1.1
const psi_1 = 0.0833
const rho_2 = 5
const psi_2 = 0.3
const JR = 45 # Retirement Age
const JJ = 80 # Death

2

In [52]:
function grid_Cons_Grow(n::Int64, left::Real, right::Real, growth::Float64)
    h = (right-left)/((1+growth)^(n-1)-1)
    # ! calculate grid value
    return [left + h*((1+growth)^i-1) for i in 0:(n-1)]
end
function grid_Cons_Grow(n::Int64, left::Real, right::Real, growth::Float64)
    h = (right-left)/((1+growth)^(n-1)-1)
    # ! calculate grid value
    return [left + h*((1+growth)^i-1) for i in 0:(n-1)]
end
function linint_Grow(x::Real, left::Real, right::Real, growth::Float64, n::Int64)
    # Input = x
    # Output = phi , il,ir 
    # invert the grid to get point
    xinv = grid_Inv_Grow(min(max(x, left), right), left, right, growth, n)  
    # get left and right gridpoint
    il = Int( min(max(floor(xinv), 1), (n-1)) ) 
    ir = il + 1
    # determine left and right gridpoint
    h = ( right-left )/( (1+growth)^(n-1) - 1 )     
    xl = h*((1+growth)^(il-1)-1) + left
    xr = h*((1+growth)^(ir-1)-1) + left
    # get share on the left point
        phi = (xr-x)/(xr-xl)
   return il, ir, phi
end
function grid_Inv_Grow(x::Real, left::Real, right::Real, growth::Float64, n::Int64)
        ##### INPUT/OUTPUT VARIABLES #############################################
    h = (right-left)/((1+growth)^(n-1) -1)
    # calculate grid value
    return log((x-left)/h+1) /log(1+growth) + 1
end 

grid_Cons_Grow (generic function with 1 method)

In [56]:
q10_GP = 0.13 ; q10_GN = 0.14
q10_BP = 0.17 ; q10_BN = 0.15
#####
q11_GP = 0.89 ; q11_GN = 0.90
q11_BP = 0.91 ; q11_BN = 0.92
#####
q20_GP = 0.02 ; q20_GN = 0.03
q20_BP = 0.04 ; q20_BN = 0.05
#####
q21_GP = 0.02 ; q21_GN = 0.04
q21_BP = 0.05 ; q21_BN = 0.06
#####
q22_GP = 0.93; q22_GN = 0.94;
q22_BP = 0.95; q22_BN = 0.96;

0.96

In [57]:
survival_prob = [0.00076,0.00086,0.00086,
0.00088,0.00094,0.00092,0.00095,
0.00093,0.00098,0.00101,0.00104,
0.00107,0.00111,0.00112,0.00115,
0.00122,0.0013,0.00138,0.00148,
0.00157,0.00165,0.0018,0.00201,
0.00215,0.00243,0.00265,0.00299,
0.00324,0.00351,0.00385,0.00422,
0.00446,0.00498,0.00529,0.00568,
0.00608,0.0066,0.00709,0.00769,
0.0081,0.00869,0.00921,0.0102,
0.01075,0.01198,0.0128,0.01414,
0.01512,0.01662,0.01789,0.0196,
0.02116,0.02336,0.02526,0.02783,
0.03093,0.03374,0.03713,0.04113,
0.04476,0.04938,0.05343,0.06043,
0.06727,0.07659,0.08675,0.09637,
0.10783,0.12186,0.13554,0.15729,
0.168,0.19137,0.21155,0.23559,
0.25584,0.28111,0.30788,0.33604,
0.36548,0.39602]
survival_prob = 1 .- survival_prob ;
survival_prob[1] = 1 ;
survival_prob[end] = 0; 

In [58]:
Age_portion = [0.016637646,0.01662503,0.016610731,0.016596433,
    0.016581798,0.016566322,0.016551014,0.016535369,
    0.016520061,0.016503744,0.016487091,0.016469932,
    0.016452438,0.016434102,0.016415766,0.016396926,
    0.016376908,0.016355712,0.016333171,0.016309116,
    0.016283378,0.016256463,0.016227193,0.016194559,
    0.016159738,0.016120543,0.016077983,0.016029873,0.015978062,
    0.015922045,0.015860813,0.015794031,0.015723715,
    0.015645662,0.015563067,0.015475088,0.015381223,0.015280123,
    0.015171959,0.015055888,0.014934434,0.014805243,
    0.01466949,0.014520785,0.014365351,0.014194441,
    0.014013775,0.013816959,0.013609546,0.013385311,
    0.013147954,0.012892767,0.012622945,0.012331422,
    0.01202375,0.011693706,0.011337587,0.010961451,
    0.010561764,0.010136171,0.009692411,0.009225268,
    0.008745173,0.008232275,0.007696499,0.007128762,
    0.00653596,0.005935083,0.005327815,0.004716004,
    0.004117315,0.003516943,0.002971747,0.002452793,
    0.001983464,0.001565441,0.001210332,0.00091208,
    0.000668668,0.000476395,0.000904006];
# Age Proportion of 2010 

In [60]:
eff1 = zeros(JJ+1)
        # initialize age earnings process
eff1[1:JR-1] = 
[1.0000, 1.0719, 1.1438, 1.2158, 1.2842, 1.3527, 
1.4212, 1.4897, 1.5582, 1.6267, 1.6952, 1.7217, 
1.7438, 1.7748, 1.8014, 1.8279, 1.8545, 1.8810, 
1.9075, 1.9341, 1.9606, 1.9623, 1.9640, 1.9658, 
1.9675, 1.9692, 1.9709, 1.9726, 1.9743, 1.9760, 
1.9777, 1.9700, 1.9623, 1.9546, 1.9469, 1.9392, 
1.9315, 1.9238, 1.9161, 1.9084, 1.9007, 1.8354, 
1.7701, 1.7048]
eff1[JR:JJ] .= 0 ;

In [61]:
eff = zeros(JJ+1)
eff[1] = 1
eff[2:JR-1] = eff1[2:JR-1] ./ eff1[1:JR-2] ;
const a_l    = 0.0
const a_u    = 150
const a_grow = 0.03
const g = 0.02
const a = grid_Cons_Grow(NA, a_l, a_u, a_grow) ;
eta , dist_eta = QuantEcon.qnwnorm(NE, 0, 1); 
δ = 0.1 ; 
s = 0.15 ;
delta = δ .+ s * eta
sort!(delta, rev = true)
const alpha = 0.36 ;
const sig2_zeta = 0.08^2;
const sig2_eps = 0.10^2;
const mu_zeta = -0.5 * sig2_zeta;
const mu_eps = -0.5 * sig2_eps;
const N_zeta = 7
const N_eps = 7
const N_zetaeps = N_zeta * N_eps
temp , dist_zetaeps = QuantEcon.qnwnorm( [N_zeta, N_eps] ,
    [mu_zeta,mu_eps], [ sig2_zeta 0 ; 0  sig2_eps] ) ;
const zeta, Eps = exp.(temp)[:,1] ,exp.(temp)[:,2] ;
Age_Por2 = zeros(JJ)
Age_Por2[1:80] = Age_portion[1:80]
Age_Por2[80] += Age_portion[81]
LL = zeros(JR-1);
LL = eff1[1:44];
a_temp = zeros(2);
u_temp = zeros(2);
c_temp = zeros(2);
LBar = LL .* Age_Por2[1:44] |> sum ;
k_max = maximum(k);
k_min = minimum(k);
X_l = minimum(Eps) * lambda
X_u = maximum(Eps) + a_u * (1+ (LBar/k_min)^(1-alpha) * alpha * 1.01 + 0.05 )
X_grow = 0.03
X = grid_Cons_Grow(NX, X_l, X_u, X_grow) ;
P = [2/3 1/3 ; 1/3 2/3]  ;
U = [1.01,0.99];

In [ ]:
# J 살 먼저 풀고 
# J-1 꺼 포트폴리오 풀기 
#=
K_next_BN
K_next_BP
K_next_GN
K_next_GP

PB_next_BN
PB_next_BP
PB_next_GN
PB_next_GP

In [ ]:
################## TYPE 1 Pre-Allocation
V_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
V_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
V_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
V_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
#########################
omega_plus_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
##########################
a_plus_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB) 
###########################
C_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
C_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
C_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
C_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
##############################
RHS_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
################################
Q_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
################################
e_plus_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB);
############# TYPE 2 Pre-Allocation
V_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
V_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
V_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
V_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB)
#########################
omega_plus_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB)
##########################
a_plus_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB) 
############################
C_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
C_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
C_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
C_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB)
##############################
RHS_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB)
################################
Q_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB);
################################
e_plus_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB);

In [30]:
using JLD2

In [31]:
JLD2.@load("Type2_2.jld2") 

28-element Array{Symbol,1}:
 :V_TYPE2_GP
 :V_TYPE2_GN
 :V_TYPE2_BP
 :V_TYPE2_BN
 :omega_plus_TYPE2_GP
 :omega_plus_TYPE2_GN
 :omega_plus_TYPE2_BP
 :omega_plus_TYPE2_BN
 :a_plus_TYPE2_GP
 :a_plus_TYPE2_GN
 :a_plus_TYPE2_BP
 :a_plus_TYPE2_BN
 :C_TYPE2_GP
 ⋮
 :RHS_TYPE2_GP
 :RHS_TYPE2_GN
 :RHS_TYPE2_BP
 :RHS_TYPE2_BN
 :Q_TYPE2_GP
 :Q_TYPE2_GN
 :Q_TYPE2_BP
 :Q_TYPE2_BN
 :e_plus_TYPE2_GP
 :e_plus_TYPE2_GN
 :e_plus_TYPE2_BP
 :e_plus_TYPE2_BN

In [45]:
################## TYPE 1 Pre-Allocation
V_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
V_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
V_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
V_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
#########################
omega_plus_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
##########################
a_plus_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB) 
###########################
C_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
C_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
C_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
C_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
##############################
RHS_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
################################
Q_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB)
################################
e_plus_TYPE1_GP = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE1_GN = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE1_BP = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE1_BN = zeros(JJ,NX,NP,NK,NPB);
############# TYPE 2 Pre-Allocation
V_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
V_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
V_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
V_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB)
#########################
omega_plus_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
omega_plus_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB)
##########################
a_plus_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
a_plus_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB) 
############################
C_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
C_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
C_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
C_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB)
##############################
RHS_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
RHS_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB)
################################
Q_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB)
Q_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB);
################################
e_plus_TYPE2_GP = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE2_GN = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE2_BP = zeros(JJ,NX,NP,NK,NPB);
e_plus_TYPE2_BN = zeros(JJ,NX,NP,NK,NPB);

In [35]:
function valuefunc(a_plus::Real, cons::Float64,ij::Int64,ip::Int64,
        iu::Int64,idep::Int64,k_next::Float64,
        PB_next::Float64,TYPE::Int64,
        ik::Int64,iPB::Int64, psi::Float64)
    # check whether consumption or leisure are too small
    c_help = max(cons, 1e-10)
    # calculate tomorrow's part of value function if ij < JJ
    ial, iar, varphi = linint_Grow(a_plus, a_l, a_u, a_grow, NA)
    v = 0 ;
    
    if ij < JJ # 내일 안 죽음 
        Q_first = bilin_Q(ij,ial,ip,iu,idep,k_next,PB_next,TYPE)
        Q_second = bilin_Q(ij,iar,ip,iu,idep,k_next,PB_next,TYPE)
        v = varphi * Q_first + (1-varphi) * Q_second
        v = max( v , 1e-10)
        return ( (1-β)*c_help^(1 - 1/psi) + β * v^(1 - 1/psi) )^( 1/(1-1/psi)) 
    else
        # 내일 죽음 
        return c_help 
    end
end 

valuefunc (generic function with 1 method)

In [36]:
#    ! solve the household's portfolio decision
function solve_portfolio(ij::Int64, ia::Int64, ip::Int64, iu::Int64, idep::Int64,
        ik::Int64,iPB::Int64,TYPE::Int64,k_next::Float64,PB_next::Float64,r_f::Float64)
        # set up communication variables
        ij_com = ij
        ia_com = ia
        ip_com = ip
        iu_com = iu
        idep_com = idep
        ik_com = ik
        iPB_com = iPB
        x_in = 0.0
    
        port0 = foc_port(0,ij_com,ia_com,ip_com,iu_com,
        idep_com,ik_com,iPB_com,TYPE,k_next,PB_next,r_f)
    
        port1 = foc_port(1,ij_com,ia_com,ip_com,iu_com,
        idep_com,ik_com,iPB_com,TYPE,k_next,PB_next,r_f)
        # use intermediate value theorem
    if ( port0*port1 > 0) # Corner Solution
        if (abs(port0) > abs(port1)) 
            return 1.0 
        else
            return 0.0
        end
    else

        #! get order of magnitude of foc

        #! get best guess for the root of foc_port

        # solve the household problem using rootfinding
        x_in = -port0/(port1-port0)
        x_in = try
            find_zero( x-> foc_port(x,ij_com,ia_com,ip_com,
                    iu_com,idep_com,ik_com,iPB_com,TYPE,k_next,PB_next,r_f), 
                x_in , atol = 0, rtol = 0) 
        catch
            find_zero( x-> foc_port(x,ij_com,ia_com,ip_com,
                    iu_com,idep_com,ik_com,iPB_com,TYPE,k_next,PB_next,r_f), 
                (0,1) , atol = 0, rtol = 0)
            end
        # write screen output in case of a problem
        # reset tolerance level to original value
    end
    if x_in <= 0
        x_in = 0.0
    elseif x_in >= 1
        x_in = 1.0
    end
    return x_in
end

solve_portfolio (generic function with 1 method)

In [ ]:
function foc_port(p::Real,ij_com::Int64,ia_com::Int64,
        ip_com::Int64,iu_com::Int64,idep_com::Int64,
        ik_com::Int64,iPB_com::Int64,TYPE::Int64,
        k_next::Float64,PB_next::Float64,r_f::Float64)
    omega_p = p
    ff = 0.0
    EV = 0.0
    rho, psi = 0,0
    if TYPE == 1
        rho = rho_1
        psi = psi_1
    else
        rho = rho_2
        psi = psi_2
    end
     
    prob = Π[iu_com,:]
    if (ij_com+1 >= JR) # 퇴직자의 문제 
        # if today is good state 
        RK_Next_BN
        RK_Next_BP
        RK_Next_GN
        RK_Next_GP
        α * U[iu] * LBar^(1-α) / k_next^(1-α) - delta[idep]
        RF_Next_BN
        RF_Next_BP
        RF_Next_GN
        RF_Next_GP
        (1-α) * U[iu] * k_next^α / LBar^α 
        W_Next_BN
        W_Next_BP
        W_Next_GN
        W_Next_GP
        
        RP_Next_BN = 1 + r_f + omega_p * (r_k - r_f)
        RP_Next_BP = 1 + r_f + omega_p * (r_k - r_f)
        RP_Next_GN = 1 + r_f + omega_p * (r_k - r_f)
        RP_Next_GP = 1 + r_f + omega_p * (r_k - r_f)
        
        X_p = R_port*a[ia_com]/(1+g)^(1/(1-α)) + lambda * w_next 
                
                ixl, ixr, varphi = linint_Grow(X_p, X_l, X_u, X_grow, NX)
                c_p_first = bilin_con(ij_com+1,ixl,ip_com,iu,idep,k_next,PB_next,TYPE)
                c_p_second = bilin_con(ij_com+1,ixr,ip_com,iu,idep,k_next,PB_next,TYPE)
                c_p = varphi * c_p_first + (1-varphi) * c_p_second
                V_p_first = bilin_V(ij_com+1,ixl,ip_com,iu,idep,k_next,PB_next, TYPE)
                V_p_second = bilin_V(ij_com+1,ixr,ip_com,iu,idep,k_next,PB_next, TYPE)
                V_p = varphi * V_p_first + (1-varphi) * V_p_second
        V_Next_BN = 
        V_Next_BP =
        V_Next_GN = 
        V_Next_GP = 
        
        dist_BN
        dist_BP
        dist_GN
        dist_GP
                dist = prob[iu] * 0.5 
        ff += dist_BN * (1+g)^(-rho/(1-α)) * (1-β)*(r_k - r_f)*a[ia_com]*
                survival_prob[ij_com+1] * V_p^(1/psi-rho)*c_p^(-1/psi)
                EV = EV + dist * survival_prob[ij_com+1] * V_p^(1-rho) 
            end
        end
    else
        for iu = 1:NU
            for idep = 1:NE 
                for ize = 1:N_zetaeps
                    r_k = α * U[iu] * LBar^(1-α) / k_next^(1-α) - delta[idep]
                    w_next = (1-α) * U[iu] * k_next^α / LBar^α 
                    earnings = w_next * Eps[ize]
                    R_port = 1 + r_f + omega_p * (r_k - r_f)
                    X_p = R_port * a[ia_com]/
                    ( eff[ij_com+1] * zeta[ize] * (1+g)^(1/(1-α)) ) + earnings
                    ixl, ixr, varphi = linint_Grow(X_p, X_l, X_u, X_grow, NX)
                    c_p_first = bilin_con(ij_com+1,ixl,ip_com,iu,idep,k_next,PB_next,TYPE)
                    c_p_second = bilin_con(ij_com+1,ixr,ip_com,iu,idep,k_next,PB_next,TYPE)
                    c_p = varphi * c_p_first + (1-varphi) * c_p_second
                    V_p_first = bilin_V(ij_com+1,ixl,ip_com,iu,idep,k_next,PB_next, TYPE)
                    V_p_second = bilin_V(ij_com+1,ixr,ip_com,iu,idep,k_next,PB_next, TYPE)
                    V_p = varphi * V_p_first + (1-varphi) * V_p_second
                    
                    dist = prob[iu] * 0.5 * dist_zetaeps[ize]
                    ff += dist*(1-β)*(eff[ij_com+1]*zeta[ize]*(1+g)^(1/(1-α)))^(-rho) * 
                        (r_k-r_f)*a[ia_com]*survival_prob[ij_com+1]*
                        V_p^(1/psi-rho)*c_p^(-1/psi)
                    EV += dist* ( eff[ij_com+1] * zeta[ize] * (1+g)^(1/(1-α)))^(1-rho) * 
                        survival_prob[ij_com+1]*V_p^(1-rho)
                end
            end
        end
    end
        
    EV = EV^(1/(1-rho))
    ff = ff*EV^rho
    return ff   
end

In [37]:
function foc_port(p::Real,ij_com::Int64,ia_com::Int64,
        ip_com::Int64,iu_com::Int64,idep_com::Int64,
        ik_com::Int64,iPB_com::Int64,TYPE::Int64,
        k_next::Float64,PB_next::Float64,r_f::Float64)
    omega_p = p
    ff = 0.0
    EV = 0.0
    rho, psi = 0,0
    if TYPE == 1
        rho = rho_1
        psi = psi_1
    else
        rho = rho_2
        psi = psi_2
    end
     
    prob = Π[iu_com,:]
    if (ij_com+1 >= JR) # 퇴직자의 문제 
        # if today is good state 
        for iu = 1:NU
            for idep = 1:NE
                
        r_k_BN
        r
                
                r_k = α * U[iu] * LBar^(1-α) / k_next^(1-α) - delta[idep]
                w_next = (1-α) * U[iu] * k_next^α / LBar^α 
                R_port = 1 + r_f + omega_p * (r_k - r_f)
                X_p = R_port*a[ia_com]/(1+g)^(1/(1-α)) + λ * w_next 
                
                ixl, ixr, varphi = linint_Grow(X_p, X_l, X_u, X_grow, NX)
                c_p_first = bilin_con(ij_com+1,ixl,ip_com,iu,idep,k_next,PB_next,TYPE)
                c_p_second = bilin_con(ij_com+1,ixr,ip_com,iu,idep,k_next,PB_next,TYPE)
                c_p = varphi * c_p_first + (1-varphi) * c_p_second
                V_p_first = bilin_V(ij_com+1,ixl,ip_com,iu,idep,k_next,PB_next, TYPE)
                V_p_second = bilin_V(ij_com+1,ixr,ip_com,iu,idep,k_next,PB_next, TYPE)
                V_p = varphi * V_p_first + (1-varphi) * V_p_second
                
                dist = prob[iu] * 0.5 
                ff = ff + dist * (1+g)^(-rho/(1-α)) * (1-β)*(r_k - r_f)*a[ia_com]*
                survival_prob[ij_com+1] * V_p^(1/psi-rho)*c_p^(-1/psi)
                EV = EV + dist * survival_prob[ij_com+1] * V_p^(1-rho) 
            end
        end
    else
        for iu = 1:NU
            for idep = 1:NE 
                for ize = 1:N_zetaeps
                    r_k = α * U[iu] * LBar^(1-α) / k_next^(1-α) - delta[idep]
                    w_next = (1-α) * U[iu] * k_next^α / LBar^α 
                    earnings = w_next * Eps[ize]
                    R_port = 1 + r_f + omega_p * (r_k - r_f)
                    X_p = R_port * a[ia_com]/
                    ( eff[ij_com+1] * zeta[ize] * (1+g)^(1/(1-α)) ) + earnings
                    ixl, ixr, varphi = linint_Grow(X_p, X_l, X_u, X_grow, NX)
                    c_p_first = bilin_con(ij_com+1,ixl,ip_com,iu,idep,k_next,PB_next,TYPE)
                    c_p_second = bilin_con(ij_com+1,ixr,ip_com,iu,idep,k_next,PB_next,TYPE)
                    c_p = varphi * c_p_first + (1-varphi) * c_p_second
                    V_p_first = bilin_V(ij_com+1,ixl,ip_com,iu,idep,k_next,PB_next, TYPE)
                    V_p_second = bilin_V(ij_com+1,ixr,ip_com,iu,idep,k_next,PB_next, TYPE)
                    V_p = varphi * V_p_first + (1-varphi) * V_p_second
                    
                    dist = prob[iu] * 0.5 * dist_zetaeps[ize]
                    ff += dist*(1-β)*(eff[ij_com+1]*zeta[ize]*(1+g)^(1/(1-α)))^(-rho) * 
                        (r_k-r_f)*a[ia_com]*survival_prob[ij_com+1]*
                        V_p^(1/psi-rho)*c_p^(-1/psi)
                    EV += dist* ( eff[ij_com+1] * zeta[ize] * (1+g)^(1/(1-α)))^(1-rho) * 
                        survival_prob[ij_com+1]*V_p^(1-rho)
                end
            end
        end
    end
        
    EV = EV^(1/(1-rho))
    ff = ff*EV^rho
    return ff   
end

foc_port (generic function with 1 method)

In [38]:
function interpolate(ij::Int64, ip::Int64, iu_com::Int64, 
        ik::Int64, iPB::Int64,TYPE::Int64, k_next::Float64,
        PB_next::Float64,omega_plus::Array{Float64,5},
        c::Array{Float64,5}, V::Array{Float64,5}, 
        RHS::Array{Float64,5},Q::Array{Float64,5})
    
    RHS[ij, :, ip,ik,iPB] .= 0
    Q[ij, :, ip,ik,iPB] .= 0
    
    prob = Π[iu_com,:]
    
    if TYPE == 1
        psi = psi_1
        rho = rho_1
    else
        psi = psi_2
        rho = rho_2
    end
    r_f = 1/PB[iPB] - 1
    # case agent is retired tomorrow
    for ia = 1:NA
        if (ij >= JR-1) 
            for iu = 1:NU
                for idep = 1:NE
                    r_k = α * U[iu] * LBar^(1-α) / k_next^(1-α) - delta[idep]
                    w_next = (1-α) * U[iu] * k_next^α / LBar^α 
                    R_port = 1 + r_f + omega_plus[ij,ia,ip,ik,iPB] * (r_k - r_f)
                    X_p = R_port*a[ia]/(1+g)^(1/(1-α)) + λ * w_next 
                
                    ixl, ixr, varphi = linint_Grow(X_p, X_l, X_u, X_grow, NX)
                    
                c_p_first = bilin_con(ij+1,ixl,ip,iu,idep,k_next,PB_next,TYPE)
                c_p_second = bilin_con(ij+1,ixr,ip,iu,idep,k_next,PB_next,TYPE)
                c_p = varphi * c_p_first + (1-varphi) * c_p_second
                V_p_first = bilin_V(ij+1,ixl,ip,iu,idep,k_next,PB_next, TYPE)
                V_p_second = bilin_V(ij+1,ixr,ip,iu,idep,k_next,PB_next, TYPE)
                EV = varphi * V_p_first + (1-varphi) * V_p_second
                        c_p = max(1e-10, c_p) 
                        
                    dist = prob[iu] * 0.5 
                    
                    RHS[ij, ia, ip, ik,iPB] += dist *  R_port * survival_prob[ij+1] * 
                        (1+g)^(-rho/(1-α)) * EV^(1/psi-rho) * c_p^(-1/psi)
                    Q[ij, ia, ip,ik,iPB] += dist * survival_prob[ij+1] * 
                        EV^(1-rho) * ( (1+g)^(1/(1-α)) )^(1-rho)
                end
            end
        else
            for iu = 1:NU
                for idep = 1:NE
                    for ize = 1:N_zetaeps
                        r_k = α * U[iu] * LBar^(1-α) / k_next^(1-α) - delta[idep]
                        w_next = (1-α) * U[iu] * k_next^α / LBar^α 
                        R_port = 1 + r_f + omega_plus[ij,ia,ip,ik,iPB] * (r_k - r_f)
                        earnings = w_next * Eps[ize]
                        X_p = R_port*a[ia]/( eff[ij+1] * zeta[ize] * (1+g)^(1/(1-α)) ) +
                            earnings
                
                        ixl, ixr, varphi = linint_Grow(X_p, X_l, X_u, X_grow, NX)
            
                        c_p_first = bilin_con(ij+1,ixl,ip,iu,idep,k_next,PB_next,TYPE)
                    c_p_second = bilin_con(ij+1,ixr,ip,iu,idep,k_next,PB_next,TYPE)
                    c_p = varphi * c_p_first + (1-varphi) * c_p_second
                    V_p_first = bilin_V(ij+1,ixl,ip,iu,idep,k_next,PB_next, TYPE)
                    V_p_second = bilin_V(ij+1,ixr,ip,iu,idep,k_next,PB_next, TYPE)
                    EV = varphi * V_p_first + (1-varphi) * V_p_second
                        c_p = max(1e-10, c_p) 
                
                        dist = prob[iu] * 0.5  * dist_zetaeps[ize]
                        # get RHS of foc and Q
                        RHS[ij, ia, ip, ik,iPB] +=  dist * R_port * 
                        ( eff[ij+1] * zeta[ize]*(1+g)^(1/(1-α)) )^(-rho) * survival_prob[ij+1] * 
                            EV^( 1/psi - rho ) * c_p^(-1/psi)
                        Q[ij, ia, ip,ik,iPB] += dist * 
                            ( zeta[ize] * eff[ij+1] * (1+g) ^ ( 1 / (1-α) ) )^(1-rho) * 
                            survival_prob[ij+1] * EV^(1-rho)
                    end
                end
            end
        end
        Q[ij,ia,ip,ik,iPB] = Q[ij,ia,ip,ik,iPB]^(1/(1-rho))
        RHS[ij,ia,ip,ik,iPB] = β^(-psi) * Q[ij,ia,ip,ik,iPB]^(1-rho*psi) * 
            RHS[ij,ia,ip,ik,iPB]^(-psi)
    end
    return RHS[ij,:,ip,ik,iPB] , Q[ij,:,ip,ik,iPB]
end

interpolate (generic function with 1 method)

In [39]:
# the first order condition regarding consumption
function foc_cons(x_in::Float64,ix_com::Int64,
        ij_com::Int64,ip_com::Int64,ip_p_com::Int64, 
        iu_com::Int64, idep_com::Int64,
        ik_com::Int64 ,iPB_com::Int64,TYPE::Int64,
        k_next::Float64,PB_next::Float64)
    # calculate tomorrows assets
    a_plus = x_in
    w_now = (1-α) * U[iu_com] * k[ik_com]^α / LBar^α 
    # calculate consumption
    # FF : Fixed Cost 
    cons_com = X[ix_com] - a_plus - (ip_p_com-ip_com)*F_cost * w_now 
    # calculate linear interpolation for future part of first order condition
    ial, iar, varphi =  linint_Grow(a_plus, a_l, a_u, a_grow, NA)
    RHS_first = bilin_RHS(ij_com,ial,ip_com,iu_com,idep_com,k_next,PB_next, TYPE)
    RHS_second = bilin_RHS(ij_com,iar,ip_com,iu_com,idep_com,k_next,PB_next, TYPE)
    tomorrow = varphi*RHS_first + (1-varphi)*RHS_second
    # calculate first order condition for consumption
    return cons_com - tomorrow
end 

foc_cons (generic function with 1 method)

In [40]:
# solve the household's consumption-savings decision
function solve_consumption(ij::Int64,ix::Int64,ip::Int64,
        ik::Int64, iPB::Int64, iu::Int64,idep::Int64,
        k_next::Float64,PB_next::Float64,
        a_plus::Array{Float64,5},e_plus::Array{Float64,5},TYPE::Int64)
    
    cons_com = 0.0
    value = 0.0
    x_in = 0.0
    psi = 0.0
    if TYPE == 1
        psi = psi_1
    else
        psi = psi_2
    end
    # determine decision for zero cash-on-hand
    if (X[ix] < 1e-10)
        return 0.0, 0.0, valuefunc(x_in,cons_com,ij,ip,
            iu,idep,k_next,PB_next,TYPE,ik,iPB,psi)
    end
    # set up communication variables
    ij_com = ij
    ix_com = ix
    ip_com = ip
    ik_com = ik
    iPB_com = iPB
    iu_com = iu
    idep_com = idep
    w_now = (1-α) * U[iu_com] * k[ik_com]^α / LBar^α 
    
    if ip == 1
        # ip == 1 : 주식시장에 아직 참여하지 않은 상태 
        # ip == 2 : 주식시장에 참여한 상태 
        for ip_p in 1:2 
            ip_p_com = ip_p
            x_temp = a_plus[ij+1,ix,ip,ik,iPB] # 초기값 
            # solve the household problem using rootfinding
            x_in =  try 
                 find_zero( x-> foc_cons(x,ix_com,ij_com,ip_com,ip_p_com,
                        iu_com,idep_com,ik_com,iPB_com,TYPE,k_next,PB_next), (a_l,X_u) )
            catch
               find_zero( x-> foc_cons(x,ix_com,ij_com,ip_com,ip_p_com,
                        iu_com,idep_com,ik_com,iPB_com,TYPE,k_next,PB_next), x_temp )
            end
            
            # check for borrowing constraint
            if x_in < 0
                x_in = 0
                cons_com = X[ix] - (ip_p_com - ip_com)*F_cost * w_now
            end
        a_temp[ip_p] = x_in
        c_temp[ip_p] = X[ix_com] - x_in - (ip_p_com-ip_com)*F_cost * w_now
        u_temp[ip_p] = valuefunc(x_in,c_temp[ip_p],ij,ip_p_com,iu,idep,k_next,PB_next,
                TYPE,ik,iPB,psi)
        end
        if  u_temp[1] > u_temp[2] 
            e_plus[ij, ix, ip, ik, iPB] = 1 # 안하기로 결정
            x_in = a_temp[1]
            cons_com = c_temp[1]
            value = u_temp[1]
            else # u_temp[1] < u_temp[2]
            e_plus[ij, ix, ip,ik,iPB] = 2 # 참가하기로 결정 
            x_in = a_temp[2]
            cons_com = c_temp[2]
            value = u_temp[2]
        end
    else # ip == 2
        ip_p_com = 2
        x_temp = a_plus[ij+1, ix, ip,ik,iPB]
        x_in =  try 
                find_zero( x-> foc_cons(x,ix_com,ij_com,ip_com,ip_p_com,
                        iu_com,idep_com,ik_com,iPB_com,TYPE,k_next,PB_next), (a_l,X_u) )
            catch
                find_zero( x-> foc_cons(x,ix_com,ij_com,ip_com,ip_p_com,
                        iu_com,idep_com,ik_com,iPB_com,TYPE,k_next,PB_next), x_temp )
            end
        if x_in < 0 
            x_in = 0.0
            cons_com = X[ix]
        end
        
        e_plus[ij,ix,ip,ik,iPB] = 2
        cons_com = X[ix] - x_in
        value = valuefunc(x_in,cons_com,ij,ip_p_com,iu,idep,k_next,PB_next,TYPE,ik,iPB,psi)
    # copy decisions
    end
    return x_in, cons_com, value
end

solve_consumption (generic function with 1 method)

In [90]:
    # get decisions in last period of life
    omega_plus_TYPE1_GP[JJ,:,:,:,:] .= 0
    omega_plus_TYPE1_GN[JJ,:,:,:,:] .= 0
    omega_plus_TYPE1_BP[JJ,:,:,:,:] .= 0
    omega_plus_TYPE1_BN[JJ,:,:,:,:] .= 0 
    
for ix in 1:NX 
# Last period에는 저축이 없음. 
    a_plus_TYPE1_GP[JJ,ix,:,:,:] .= 0
    a_plus_TYPE1_GN[JJ,ix,:,:,:] .= 0
    a_plus_TYPE1_BP[JJ,ix,:,:,:] .= 0
    a_plus_TYPE1_BN[JJ,ix,:,:,:] .= 0
    for ik = 1:NK
        for iPB = 1:NPB
            for ip = 1:2
                C_TYPE1_GP[JJ,ix,ip,ik,iPB] = X[ix]
                C_TYPE1_GN[JJ,ix,ip,ik,iPB] = X[ix]
                C_TYPE1_BP[JJ,ix,ip,ik,iPB] = X[ix]
                C_TYPE1_BN[JJ,ix,ip,ik,iPB] = X[ix]
                V_TYPE1_GP[JJ,ix,ip,ik,iPB] = C_TYPE1_GP[JJ,ix,ip,ik,iPB]
                V_TYPE1_GN[JJ,ix,ip,ik,iPB] = C_TYPE1_GN[JJ,ix,ip,ik,iPB]
                V_TYPE1_BP[JJ,ix,ip,ik,iPB] = C_TYPE1_BP[JJ,ix,ip,ik,iPB]
                V_TYPE1_BN[JJ,ix,ip,ik,iPB] = C_TYPE1_BN[JJ,ix,ip,ik,iPB]
            end
        end
    end
end

    # get decisions in last period of life
    omega_plus_TYPE2_GP[JJ,:,:,:,:] .= 0
    omega_plus_TYPE2_GN[JJ,:,:,:,:] .= 0
    omega_plus_TYPE2_BP[JJ,:,:,:,:] .= 0
    omega_plus_TYPE2_BN[JJ,:,:,:,:] .= 0 
    
for ix in 1:NX 
# Last period에는 저축이 없음. 
    a_plus_TYPE2_GP[JJ,ix,:,:,:] .= 0
    a_plus_TYPE2_GN[JJ,ix,:,:,:] .= 0
    a_plus_TYPE2_BP[JJ,ix,:,:,:] .= 0
    a_plus_TYPE2_BN[JJ,ix,:,:,:] .= 0
    for ik = 1:NK
        for iPB = 1:NPB
            for ip = 1:2
                C_TYPE2_GP[JJ,ix,ip,ik,iPB] = X[ix]
                C_TYPE2_GN[JJ,ix,ip,ik,iPB] = X[ix]
                C_TYPE2_BP[JJ,ix,ip,ik,iPB] = X[ix]
                C_TYPE2_BN[JJ,ix,ip,ik,iPB] = X[ix]
                V_TYPE2_GP[JJ,ix,ip,ik,iPB] = C_TYPE2_GP[JJ,ix,ip,ik,iPB]
                V_TYPE2_GN[JJ,ix,ip,ik,iPB] = C_TYPE2_GN[JJ,ix,ip,ik,iPB]
                V_TYPE2_BP[JJ,ix,ip,ik,iPB] = C_TYPE2_BP[JJ,ix,ip,ik,iPB]
                V_TYPE2_BN[JJ,ix,ip,ik,iPB] = C_TYPE2_BN[JJ,ix,ip,ik,iPB]
            end
        end
    end
end

In [68]:
TYPE = 2
for ij = 42:-1:34
    # determine optimal portfolio choice for all others
    println("Age $ij Start")
    for iu = 1:NU
        for idep = 1:NE
            if  iu == 1 && idep == 1 && TYPE == 1
                q10 = q10_GP; q11 = q11_GP;
                q20 = q20_GP; q21 = q21_GP;
                q22 = q22_GP
                omega_plus = omega_plus_TYPE1_GP
                RHS = RHS_TYPE1_GP
                Q = Q_TYPE1_GP
                a_plus = a_plus_TYPE1_GP
                V = V_TYPE1_GP
                C = C_TYPE1_GP
                e_plus = e_plus_TYPE1_GP
            elseif iu == 1 && idep == 2 && TYPE == 1
                q10 = q10_GN; q11 = q11_GN;
                q20 = q20_GN; q21 = q21_GN;
                q22 = q22_GN
                omega_plus = omega_plus_TYPE1_GN
                RHS = RHS_TYPE1_GN
                Q = Q_TYPE1_GN
                a_plus = a_plus_TYPE1_GN
                V = V_TYPE1_GN
                C = C_TYPE1_GN
                e_plus = e_plus_TYPE1_GN
            elseif iu == 2 && idep == 1 && TYPE == 1
                q10 = q10_BP; q11 = q11_BP;
                q20 = q20_BP; q21 = q21_BP;
                q22 = q22_BP
                omega_plus = omega_plus_TYPE1_BP
                RHS = RHS_TYPE1_BP
                Q = Q_TYPE1_BP
                a_plus = a_plus_TYPE1_BP
                V = V_TYPE1_BP
                C = C_TYPE1_BP
                e_plus = e_plus_TYPE1_BP
            elseif iu == 2 && idep == 2 && TYPE == 1
                q10 = q10_BN; q11 = q11_BN;
                q20 = q20_BN; q21 = q21_BN;
                q22 = q22_BN
                omega_plus = omega_plus_TYPE1_BN
                RHS = RHS_TYPE1_BN
                Q = Q_TYPE1_BN
                a_plus = a_plus_TYPE1_BN
                V = V_TYPE1_BN
                C = C_TYPE1_BN
                e_plus = e_plus_TYPE1_BN
            elseif iu == 1 && idep == 1 && TYPE == 2
                q10 = q10_GP; q11 = q11_GP;
                q20 = q20_GP; q21 = q21_GP;
                q22 = q22_GP
                omega_plus = omega_plus_TYPE2_GP
                RHS = RHS_TYPE2_GP
                Q = Q_TYPE2_GP
                a_plus = a_plus_TYPE2_GP
                V = V_TYPE2_GP
                C = C_TYPE2_GP
                e_plus = e_plus_TYPE2_GP
            elseif iu == 1 && idep == 2 && TYPE == 2
                q10 = q10_GN; q11 = q11_GN;
                q20 = q20_GN; q21 = q21_GN;
                q22 = q22_GN
                omega_plus = omega_plus_TYPE2_GN
                RHS = RHS_TYPE2_GN
                Q = Q_TYPE2_GN
                a_plus = a_plus_TYPE2_GN
                V = V_TYPE2_GN
                C = C_TYPE2_GN
                e_plus = e_plus_TYPE2_GN
            elseif iu == 2 && idep == 1 && TYPE == 2
                q10 = q10_BP; q11 = q11_BP;
                q20 = q20_BP; q21 = q21_BP;
                q22 = q22_BP
                omega_plus = omega_plus_TYPE2_BP
                RHS = RHS_TYPE2_BP
                Q = Q_TYPE2_BP
                a_plus = a_plus_TYPE2_BP
                V = V_TYPE2_BP
                C = C_TYPE2_BP
                e_plus = e_plus_TYPE2_BP
            else
                q10 = q10_BN; q11 = q11_BN;
                q20 = q20_BN; q21 = q21_BN;
                q22 = q22_BN
                omega_plus = omega_plus_TYPE2_BN
                RHS = RHS_TYPE2_BN
                Q = Q_TYPE2_BN
                a_plus = a_plus_TYPE2_BN
                V = V_TYPE2_BN
                C = C_TYPE2_BN
                e_plus = e_plus_TYPE2_BN
            end
            
            for ik = 1:NK
                for iPB = 1:NPB
                    r_f = 1/PB[iPB] - 1
                    k_next = q10 + q11*log(k[ik]) |> exp
                    PB_next = q20 + q21*log(k[ik]) + q22*log(PB[iPB]) |> exp 
                    for ia = 2:NA
                        omega_plus[ij,ia,2,ik,iPB] = 
                        solve_portfolio(ij,ia,2,iu,idep,ik,iPB,TYPE,k_next,PB_next,r_f)
                    end

                    # set omega for zero savings consistent with next gridpoint
                    omega_plus[ij,1,2,ik,iPB] = omega_plus[ij,2,2,ik,iPB]
                    omega_plus[ij,:,1,:,:] .= 0
                end
            end
            println("Part1 End")
            # interpolate individual RHS and value function
            println("Part2 Start")
            for ik = 1:NK
                for iPB = 1:NPB
                    r_f = 1/PB[iPB] - 1
                    k_next = q10 + q11*log(k[ik]) |> exp
                    PB_next = q20 + q21*log(k[ik]) + q22*log(PB[iPB]) |> exp 
                    for ip = 1:NP
                        RHS[ij,:,ip,ik,iPB], Q[ij,:,ip,ik,iPB] = 
                            interpolate(ij,ip,iu,ik,iPB,TYPE,k_next,PB_next,
                                omega_plus,C,V,RHS,Q)
                    end
                end
            end
            println("Part 2 End")
            # determine consumption-savings solution
            println("Part3 Start")
            for ik = 1:NK
                for iPB = 1:NPB
                k_next = q10 + q11*log(k[ik]) |> exp
                    PB_next = q20 + q21*log(k[ik]) + q22*log(PB[iPB]) |> exp 
                    for ip = 1:2 
                        for ix = 1:NX
                            a_plus[ij,ix,ip,ik,iPB], C[ij,ix,ip,ik,iPB], V[ij,ix,ip,ik,iPB] = 
                                solve_consumption(ij,ix,ip,ik,iPB,iu,idep,k_next,PB_next,
                                a_plus,e_plus,TYPE)
                        end
                    end
                end
            end
            println("Part3 End")
        end
    end
    println("Age $ij is Done")
end
    


Age 42 Start
Part1 End
Part2 Start
Part 2 End
Part3 Start
Part3 End
Part1 End
Part2 Start
Part 2 End
Part3 Start
Part3 End
Part1 End
Part2 Start
Part 2 End
Part3 Start
Part3 End
Part1 End
Part2 Start
Part 2 End
Part3 Start
Part3 End
Age 42 is Done
Age 41 Start
Part1 End
Part2 Start
Part 2 End
Part3 Start
Part3 End
Part1 End
Part2 Start
Part 2 End
Part3 Start
Part3 End
Part1 End
Part2 Start
Part 2 End
Part3 Start
Part3 End
Part1 End
Part2 Start
Part 2 End
Part3 Start
Part3 End
Age 41 is Done
Age 40 Start
Part1 End
Part2 Start


LoadError: [91mInterruptException:[39m

In [69]:
using JLD2

In [70]:
@save "Type2_2.jld2" V_TYPE2_GP  V_TYPE2_GN V_TYPE2_BP V_TYPE2_BN omega_plus_TYPE2_GP omega_plus_TYPE2_GN omega_plus_TYPE2_BP omega_plus_TYPE2_BN a_plus_TYPE2_GP a_plus_TYPE2_GN a_plus_TYPE2_BP a_plus_TYPE2_BN C_TYPE2_GP C_TYPE2_GN C_TYPE2_BP C_TYPE2_BN RHS_TYPE2_GP RHS_TYPE2_GN RHS_TYPE2_BP RHS_TYPE2_BN Q_TYPE2_GP Q_TYPE2_GN Q_TYPE2_BP Q_TYPE2_BN e_plus_TYPE2_GP e_plus_TYPE2_GN e_plus_TYPE2_BP e_plus_TYPE2_BN 

In [72]:
omega_plus_TYPE2_GP[79,:,2,:,:]

51×16×25 Array{Float64,3}:
[:, :, 1] =
 0.145239  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.145239  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.128407  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.122797  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.119993  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.118312  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.117192  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.116393  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.115794  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.115329  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.114957  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.114653  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.1144    0.0  0.0  0.0 

In [73]:
omega_plus_TYPE2_GN[79,:,2,:,:]

51×16×25 Array{Float64,3}:
[:, :, 1] =
 0.114041   0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.114041   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.100747   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0963172  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.094103   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0927752  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0918907  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0912594  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0907864  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.090419   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0901254  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0898855  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.089686   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0